# Graphcore Poplar Path-Tracer on Gradient

This notebook contains the instructions to build and run a [Poplar C++ Ray/Path Tracer](https://github.com/markp-gc/ipu_ray_lib) for Graphcore IPUs. This notebook demonstrates how to render a path-traced image and also how analyse the correctness of the ray-tracer by comparing its output with [Embree](https://www.embree.org).

![Suzanne Path-Traced on IPU](images/monkey_bust_16384spp.png)

## Instructions

First you need to launch an IPU machine: click the 'start machine' button above.

![Start Machine Screenshot](images/start_machine.png)

### Clone and Build the Code

We can execute these shell commands to build the application. The build uses CMake:

In [ ]:
!cd ipu_ray_lib && git pull
!mkdir -p ipu_ray_lib/build
!cd ipu_ray_lib/build && cmake -G Ninja .. && ninja -j64

### Run the Application

The application loads mesh data using the [Open Asset Import Library](https://github.com/assimp/assimp).
Currently meshes need to fit on tile, the provided mesh is small enough. Execute the following cell: after ~30
seconds it will output an EXR image:

In [ ]:
!cd ipu_ray_lib/build && ./test -w 1440 -h 1440 --mesh-file ../assets/monkey_bust.glb --render-mode path-trace --visualise rgb --samples 1000 --ipus 4 --ipu-only

The resulting image is high dynamic range (HDR) in EXR format. We can quickly tonemap and display the
resulting image in Python:

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

EXR_FLAGS = cv2.IMREAD_UNCHANGED | cv2.IMREAD_ANYCOLOR | cv2.IMREAD_ANYDEPTH
hdr = cv2.imread('ipu_ray_lib/build/out_rgb_ipu.exr', EXR_FLAGS)
print(f"HDR image shape: {hdr.shape} type: {hdr.dtype} min: {np.min(hdr)} max: {np.max(hdr)}")
exposure = 1.2
gamma = 2.4
scale = 2.0 ** exposure
ldr = np.power(hdr * scale, 1.0 / gamma) * 255
ldr = np.clip(ldr, 0, 255)
plt.figure(figsize=(8, 8))
plt.style.use('dark_background')
plt.imshow(cv2.cvtColor(ldr.astype(np.uint8), cv2.COLOR_BGR2RGB), interpolation='bicubic')
plt.show()
cv2.imwrite('tonemapped.png', ldr)

You can open or download the saved result [tonemapped.png](tonemapped.png) in the file browser on the left. You may have to click on the refresh icon to update the view of the files.

If you want to render a CPU reference image remove the option `--ipu-only` but be aware it will take
much much longer to render. (For a list of all command options run `./test --help`.)

If you just want to compare AOVs between CPU/Embree/IPU you can
change to a quicker render mode. For example to compare normals run this:

In [ ]:
!cd ipu_ray_lib/build && ./test -w 1440 -h 1440 --mesh-file ../assets/monkey_bust.glb --render-mode shadow-trace --visualise normal --ipus 4

Once the outputs are ready we can load them into Python to compare:

In [ ]:
# Load normals:
ipu_normals = cv2.imread('ipu_ray_lib/build/out_normal_ipu.exr', EXR_FLAGS)
embree_normals = cv2.imread('ipu_ray_lib/build/out_normal_embree.exr', EXR_FLAGS)
abs_err = np.abs(ipu_normals - embree_normals)
print(f"IPU normals min: {np.min(ipu_normals)} max: {np.max(ipu_normals)}")
print(f"Embree normals min: {np.min(embree_normals)} max: {np.max(embree_normals)}")
print(f"ABS Error min: {np.min(abs_err)} max: {np.max(abs_err)} mean: {np.mean(abs_err)}")

# Plot them side by side:
vis_ipu = ((ipu_normals + 1.0) / 2.0)
vis_embree = ((embree_normals + 1.0) / 2.0)
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(16, 8))
ax[0].imshow(vis_ipu)
ax[0].set_title('IPU')
ax[1].imshow(vis_embree)
ax[1].set_title('Embree')
plt.show()

### Enjoy

Although the program is currently limited in the size of BVH it can render it is very fast due to the IPU's large scale mulitple-instruction-multiple-data (MIMD) parallelism. For experimenting and/or learning about path-tracing you can render high resolution results much faster than you can on CPU with the same style of C++ code. For example the IPU path-trace kernel's inner loop is nothing fancy, it looks like this:
```C++
  for (auto i = 0u; i < maxPathLength; ++i) {
    // Reset ray limits for next bounce:
    hit.r.tMin = rayOffset; // offset rays to avoid self intersection.
    hit.r.tMax = std::numeric_limits<float>::infinity();
    auto intersected = bvh.intersect(hit.r, primLookup);

    if (intersected) {
      updateHit(intersected, hit);
      const auto& material = wrappedMaterials[wrappedMatIDs[hit.geomID]];

      if (material.emissive) {
        color += throughput * material.emission;
      }

      if (material.type == Material::Type::Diffuse) {
        // Use HW random number generator for samples:
        const float u1 = hw_uniform_0_1();
        const float u2 = hw_uniform_0_1();
        hit.r.direction = sampleDiffuse(hit.normal, u1, u2);
        // Update throughput
        throughput *= material.albedo;
      } else if (material.type == Material::Type::Specular) {
        hit.r.direction = reflect(hit.r.direction, hit.normal);
        throughput *= material.albedo;
      } else if (material.type == Material::Type::Refractive) {
        const float u1 = hw_uniform_0_1();
        constexpr float refractiveIndex = 1.52f;
        hit.r.direction = refract(hit.r, hit.normal, refractiveIndex, u1);
        throughput *= material.albedo * (refractiveIndex);
      }
    } else {
      break;
    }

    // Random stopping:
    if (i > rouletteStartDepth) {
      const float u1 = hw_uniform_0_1();
      if (evaluateRoulette(u1, throughput)) { break; }
    }
  }
```

If you want to browse the code here are some good starting points:

- [TraceCodelets.cpp](ipu_ray_lib/codelets/TraceCodelets.cpp): IPU ray-tracing and path-tracing C++ kernels.
- [test.cpp](ipu_ray_lib/test.cpp): The main program. In particular note the functions: `renderEmbree`, `renderCPU`, `renderIPU`.
- [IpuScene.cpp](ipu_ray_lib/src/IpuScene.cpp): This compiles the IPU ray/path trace graph program using the Poplar graph compiler.
- [README.md](ipu_ray_lib/README.md): Contains more information about how the program works and its origins.

If you want to make significant changes then you will need to consult the [Poplar SDK documentation](https://docs.graphcore.ai/projects/poplar-user-guide/en/latest/introduction.html).

If you want more speed you can try a BOW-POD (where each chip is clocked
40% higher than a standard POD).